# Tweet Generation using NLP

## Importing necessary libraries

In [182]:
import pandas as pd
import re
import csv
!pip install keras
!pip install nltk
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.stem import PorterStemmer

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [183]:
nltk.download("stopwords")
nltk.download("punkt")
nltk.download("wordnet")

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

## Dataset Reading

In [184]:
# Opening Sentiment140 dataset in text mode to handle encoding errors
with open('Sentiment140.csv', 'rt',encoding='ISO-8859-1') as f:
    reader = csv.reader(f, delimiter=',', quotechar='"')

    # Skipping the problematic lines and creating a new list of valid rows
    rows = []
    for i, row in enumerate(reader):
        if len(row) == 6:
            rows.append(row)
        else:
            print(f'Skipping row {i}: {row}')

df = pd.DataFrame(rows, columns=['target', 'id', 'date', 'flag', 'user', 'text'])
df.head()



,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Considering any one user - ElleCTF

In [185]:
df.loc[df['user']=="ElleCTF"]

,target,id,date,flag,user,text
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
120839,0,1833373892,Sun May 17 23:05:15 PDT 2009,NO_QUERY,ElleCTF,Killlll me... I can't believe I have the flu
356398,0,2044263917,Fri Jun 05 09:06:19 PDT 2009,NO_QUERY,ElleCTF,"@kennypistol it hates me today, first it would..."
359250,0,2045682914,Fri Jun 05 11:05:01 PDT 2009,NO_QUERY,ElleCTF,"On the jurassic park ride, think its gonna rai..."
359639,0,2045915728,Fri Jun 05 11:25:34 PDT 2009,NO_QUERY,ElleCTF,Alright the rides have been suspended bc of th...
457329,0,2071542055,Sun Jun 07 19:33:54 PDT 2009,NO_QUERY,ElleCTF,"Vacation in 2 weeks, Ill be gone for 10 days. ..."
655920,0,2240196468,Fri Jun 19 09:27:21 PDT 2009,NO_QUERY,ElleCTF,@passionshaker seriously? I never got it I'm ...
663643,0,2244206853,Fri Jun 19 14:26:37 PDT 2009,NO_QUERY,ElleCTF,"@meganctf well, technically Its not a TRUE job..."
683104,0,2250179207,Fri Jun 19 23:53:26 PDT 2009,NO_QUERY,ElleCTF,@hurricanehalvo omg wait is it still alive!? i...
748155,0,2284143377,Mon Jun 22 13:31:04 PDT 2009,NO_QUERY,ElleCTF,@meganctf GET ME SOME CREEPER POSTCARDS!!! Wri...


## Data Cleaning

In [186]:
# Dropping any rows with empty content
df.dropna(subset=['text'], inplace=True)

# Dropping duplicate entries
df.drop_duplicates(subset=['text'], inplace=True)

# Resetting index of the dataframe
df.reset_index(inplace=True, drop=True)

df.head()

,target,id,date,flag,user,text
0,0,1467810369,Mon Apr 06 22:19:45 PDT 2009,NO_QUERY,_TheSpecialOne_,"@switchfoot http://twitpic.com/2y1zl - Awww, t..."
1,0,1467810672,Mon Apr 06 22:19:49 PDT 2009,NO_QUERY,scotthamilton,is upset that he can't update his Facebook by ...
2,0,1467810917,Mon Apr 06 22:19:53 PDT 2009,NO_QUERY,mattycus,@Kenichan I dived many times for the ball. Man...
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
4,0,1467811193,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,Karoli,"@nationwideclass no, it's not behaving at all...."


## Data Preprocessing

In [187]:
# Filtering tweets by a particular user
target_user = "ElleCTF"
user_tweets = df[df["user"] == target_user]["text"].values
print("User tweets=", user_tweets)

stop_words = set(stopwords.words("english"))
lemmatizer = WordNetLemmatizer()
stemmer=PorterStemmer()

def preprocess_tweet(tweet):
    # Removing URLs
    tweet = re.sub(r"http\S+", "", tweet)

    # Removing mentions
    tweet = re.sub(r"@[^\s]+", "", tweet)

    # Removing hashtags
    tweet = re.sub(r"#", "", tweet)

    # Tokenizing tweet
    tokens = word_tokenize(tweet.lower())

    # Removing stop words and punctuation
    tokens = [token for token in tokens if token not in stop_words and token.isalpha()]

    # Lemmatizing tokens
    tokens = [lemmatizer.lemmatize(token) for token in tokens]
    
    # Stemming tokens
    tokens = [stemmer.stem(token) for token in tokens]

    # Joining tokens to form tweet
    tweet = " ".join(tokens)

    return tweet

# Apply preprocessing to tweets
preprocessed_tweets = [preprocess_tweet(tweet) for tweet in user_tweets]

preprocessed_tweets

User tweets= ['my whole body feels itchy and like its on fire '
 "Killlll me... I can't believe I have the flu "
 "@kennypistol it hates me today, first it wouldn't verify my pass, then the mummy is closed, and now wolverine isn't out here  and its hot"
 'On the jurassic park ride, think its gonna rain. Dnw to be on the edge  second row. Fml I just started to dry off'
 'Alright the rides have been suspended bc of the rain. I am sad '
 'Vacation in 2 weeks, Ill be gone for 10 days. Thats the longest me and megan will have gone without seeing each other since 8th grade '
 "@passionshaker seriously? I never got it  I'm sorry, my computer/phone is an asswhore I'm not mad though, takes a lot to piss me off"
 '@meganctf well, technically Its not a TRUE job. I do it because my corner got shut down  bad times and all'
 '@hurricanehalvo omg wait is it still alive!? im scared for you  lol you are what you have, thats what I tell people :]'
 '@meganctf GET ME SOME CREEPER POSTCARDS!!! Write to me

['whole bodi feel itchi like fire',
 'killlll ca believ flu',
 'hate today first would verifi pa mummi close wolverin hot',
 'jurass park ride think gon na rain dnw edg second row fml start dri',
 'alright ride suspend bc rain sad',
 'vacat week ill gone day that longest megan gone without see sinc grade',
 'serious never got sorri asswhor mad though take lot piss',
 'well technic true job corner got shut bad time',
 'omg wait still aliv im scare lol that tell peopl',
 'get creeper postcard write imi alreadi',
 'cameron would come lake send postcard make hot like fat guy thong someth',
 'clue got problem bite itch till bleed noth help one know wrong']

## Sequence Tokenization

In [188]:
from keras.preprocessing.text import Tokenizer
import tensorflow

tokenizer = Tokenizer()
tokenizer.fit_on_texts(preprocessed_tweets)
sequences = tokenizer.texts_to_sequences(preprocessed_tweets)
print("tokenizer.word_index=",tokenizer.word_index)
# Padding sequences
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_sequence_length = max([len(seq) for seq in sequences])
sequences_padded = pad_sequences(sequences, maxlen=max_sequence_length, padding="pre")

vocab_size = len(tokenizer.word_index) + 1

# Model
from keras.models import Sequential
from keras.layers import Embedding, LSTM, Dense

model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_length-1))
model.add(LSTM(150))
model.add(Dense(vocab_size, activation='softmax'))

model.compile(loss='categorical_crossentropy', optimizer='adam')

# Training the model
from keras.utils import to_categorical

X = sequences_padded[:, :-1]
y = to_categorical(sequences_padded[:, -1], num_classes=vocab_size)
model.fit(X, y, epochs=100, verbose=2)

model.save('model.h5')

tokenizer.word_index= {'got': 1, 'like': 2, 'would': 3, 'hot': 4, 'ride': 5, 'rain': 6, 'gone': 7, 'that': 8, 'postcard': 9, 'whole': 10, 'bodi': 11, 'feel': 12, 'itchi': 13, 'fire': 14, 'killlll': 15, 'ca': 16, 'believ': 17, 'flu': 18, 'hate': 19, 'today': 20, 'first': 21, 'verifi': 22, 'pa': 23, 'mummi': 24, 'close': 25, 'wolverin': 26, 'jurass': 27, 'park': 28, 'think': 29, 'gon': 30, 'na': 31, 'dnw': 32, 'edg': 33, 'second': 34, 'row': 35, 'fml': 36, 'start': 37, 'dri': 38, 'alright': 39, 'suspend': 40, 'bc': 41, 'sad': 42, 'vacat': 43, 'week': 44, 'ill': 45, 'day': 46, 'longest': 47, 'megan': 48, 'without': 49, 'see': 50, 'sinc': 51, 'grade': 52, 'serious': 53, 'never': 54, 'sorri': 55, 'asswhor': 56, 'mad': 57, 'though': 58, 'take': 59, 'lot': 60, 'piss': 61, 'well': 62, 'technic': 63, 'true': 64, 'job': 65, 'corner': 66, 'shut': 67, 'bad': 68, 'time': 69, 'omg': 70, 'wait': 71, 'still': 72, 'aliv': 73, 'im': 74, 'scare': 75, 'lol': 76, 'tell': 77, 'peopl': 78, 'get': 79, 'creepe

## Function for tweet generation

In [189]:
import numpy as np

def generate_tweet(model, seed_text, max_sequence_length, tokenizer, temperature):
    # Converting seed text to sequence
    seed_seq = tokenizer.texts_to_sequences([seed_text])[0]
    print("seed_seq=",seed_seq)
    # Padding sequence
    seed_seq = pad_sequences([seed_seq], maxlen=max_sequence_length-1, padding="pre")
    print("seed_seq after padding=",seed_seq)
    # Initializing generated tweet with seed text
    generated_tweet = seed_text
    
    # Generating next words using model and seed sequence
    for i in range(max_sequence_length-len(seed_seq)):
        preds = model.predict(seed_seq, verbose=0)[0]
        print("preds=", preds)
        # Applying temperature to adjust randomness
        preds = np.log(preds) / temperature
        exp_preds = np.exp(preds)
        preds = exp_preds / np.sum(exp_preds)
        
        # Sample from probability distribution to get next word
        next_word_idx = np.random.choice(len(preds), p=preds)
        while next_word_idx+1 not in tokenizer.index_word:
          next_word_idx = np.random.choice(len(preds), p=preds)
        next_word = tokenizer.index_word[next_word_idx+1]
        print("next_word=",next_word)
        # Adding next word to generated tweet
        generated_tweet += " " + next_word
        print("generated_tweet=",generated_tweet)
        # Updating seed sequence with next word
        seed_seq = np.append(seed_seq[:, 1:], [[next_word_idx]], axis=1)
        print("seed_seq after update=",seed_seq)
    return generated_tweet, np.array(preds)


## Model Path

In [190]:
import os

model_filename = "model.h5"
model_path = os.path.abspath(model_filename)

print("Actual file path of pre-trained model: ", model_path)


Actual file path of pre-trained model:  /content/model.h5


## Loading the model

In [191]:
from keras.models import load_model

model = load_model(model_path)


In [192]:
df.loc[df['user']=="ElleCTF"]

,target,id,date,flag,user,text
3,0,1467811184,Mon Apr 06 22:19:57 PDT 2009,NO_QUERY,ElleCTF,my whole body feels itchy and like its on fire
119746,0,1833373892,Sun May 17 23:05:15 PDT 2009,NO_QUERY,ElleCTF,Killlll me... I can't believe I have the flu
352691,0,2044263917,Fri Jun 05 09:06:19 PDT 2009,NO_QUERY,ElleCTF,"@kennypistol it hates me today, first it would..."
355513,0,2045682914,Fri Jun 05 11:05:01 PDT 2009,NO_QUERY,ElleCTF,"On the jurassic park ride, think its gonna rai..."
355899,0,2045915728,Fri Jun 05 11:25:34 PDT 2009,NO_QUERY,ElleCTF,Alright the rides have been suspended bc of th...
452325,0,2071542055,Sun Jun 07 19:33:54 PDT 2009,NO_QUERY,ElleCTF,"Vacation in 2 weeks, Ill be gone for 10 days. ..."
648238,0,2240196468,Fri Jun 19 09:27:21 PDT 2009,NO_QUERY,ElleCTF,@passionshaker seriously? I never got it I'm ...
655874,0,2244206853,Fri Jun 19 14:26:37 PDT 2009,NO_QUERY,ElleCTF,"@meganctf well, technically Its not a TRUE job..."
675109,0,2250179207,Fri Jun 19 23:53:26 PDT 2009,NO_QUERY,ElleCTF,@hurricanehalvo omg wait is it still alive!? i...
739141,0,2284143377,Mon Jun 22 13:31:04 PDT 2009,NO_QUERY,ElleCTF,@meganctf GET ME SOME CREEPER POSTCARDS!!! Wri...


## Tweet Generation

In [193]:
# Seed text is usually taken as the phrase with which the user may want to start the tweet
seed_text = "my whole body"
new_tweet, preds = generate_tweet(model, seed_text, max_sequence_length, tokenizer, temperature=0.4)

print("Generated tweet: ", new_tweet)


seed_seq= [10]
seed_seq after padding= [[ 0  0  0  0  0  0  0  0  0  0  0  0 10]]
preds= [9.99902477e-06 1.87199257e-05 8.28069642e-06 2.15420860e-05
 1.86880690e-03 6.01389365e-06 3.67893554e-05 2.24463311e-05
 9.67599226e-06 6.08792379e-06 2.88840092e-05 3.65421402e-05
 1.64451558e-05 8.79121490e-06 1.74351856e-01 7.47017111e-06
 9.73910301e-06 7.15023498e-06 4.85119671e-01 1.57144823e-05
 9.98608357e-06 4.06752988e-05 1.99718907e-05 7.93397248e-06
 1.01244377e-05 3.30602670e-05 1.40455495e-05 8.80994867e-06
 1.21422490e-05 1.48723966e-05 1.33789363e-05 1.22631718e-05
 5.87783234e-06 3.97128242e-05 1.35548962e-05 1.62344786e-05
 1.50544111e-05 1.30181488e-05 3.99588927e-04 2.97937822e-05
 2.48854958e-05 7.42385828e-06 1.76563174e-01 7.74396176e-06
 6.79465984e-06 1.92879670e-05 4.57447277e-05 1.39352233e-05
 1.29613627e-05 1.52049024e-05 1.48564923e-05 6.78069046e-06
 6.43185864e-04 6.23453889e-05 1.97532972e-05 1.40560287e-05
 1.27426729e-05 6.51102118e-06 1.08725699e-05 1.16185129e

## Metrics calculation

## Function for perplexity

In [194]:
def calculate_perplexity(generated_tweets, reference_tweets, preds):
    total_log_prob = 0.0
    num_words = 0

    for i in range(len(generated_tweets)):
        generated_sequence = generated_tweets[i].split()

        if i < len(reference_tweets):  # Checking if index is within range
            reference_sequence = reference_tweets[i].split()

            for j in range(len(generated_sequence)):
                word = generated_sequence[j]
                if word in tokenizer.word_index:
                    word_idx = tokenizer.word_index[word]
                    log_prob = np.log(preds[word_idx])
                    total_log_prob += log_prob
                    num_words += 1

    avg_log_prob = total_log_prob / num_words
    perplexity = np.exp(-avg_log_prob)

    return perplexity


## Function for BLEU Score

In [195]:
import nltk
from nltk.translate.bleu_score import sentence_bleu, corpus_bleu


def calculate_bleu_score(generated_tweets, reference_tweets):
    # Calculate BLEU score using NLTK library
    bleu_scores = []
    for gen_tweet, ref_tweet in zip(generated_tweets, reference_tweets):
        gen_tweet = gen_tweet.split()
        ref_tweet = [ref_tweet.split()]
        bleu_score = sentence_bleu(ref_tweet, gen_tweet)
        bleu_scores.append(bleu_score)
    avg_bleu_score = sum(bleu_scores) / len(bleu_scores)
    return avg_bleu_score



In [196]:
generated_tweets = new_tweet.split()
reference_tweets=user_tweets
# Evaluating perplexity
perplexity = calculate_perplexity(generated_tweets, reference_tweets,preds)
print("Perplexity:", perplexity)

# Evaluating BLEU score
bleu_score = calculate_bleu_score(generated_tweets, reference_tweets)
print("BLEU Score:", bleu_score)



Perplexity: 2747208774.45528
BLEU Score: 1.8735994074691278e-236


/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 2-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 3-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_msg)
/usr/local/lib/python3.10/dist-packages/nltk/translate/bleu_score.py:552: UserWarning: 
The hypothesis contains 0 counts of 4-gram overlaps.
Therefore the BLEU score evaluates to 0, independently of
how many N-gram overlaps of lower order it contains.
Consider using lower n-gram order or use SmoothingFunction()
  warnings.warn(_